In [1]:
%load_ext autoreload
%autoreload 2

# Imports and definitions

In [2]:
from datetime import datetime
import xarray as xr
from pathlib import Path

In [3]:
import ecephys_analyses as ea
from sglxarray import load_trigger
from ecephys.xrsig import parallel_spectrogram_welch

In [4]:
xr.set_options(keep_attrs=True)

In [5]:
def get_spectrogram(sig, **kwargs):
    nperseg = int(4 * sig.fs) # 4 second window
    noverlap = nperseg // 4 # 1 second overlap
    spg = parallel_spectrogram_welch(sig, nperseg=nperseg, noverlap=noverlap, **kwargs)
    return spg.median(dim="channel").sel(frequency=slice(0, 300))

In [8]:
def get_condition_spgs(subject, experiment, alias, probe):
    cx_chans = ea.get_channels(subject, experiment, probe, "superficial_ctx", asarray=False)
    wm_chans = ea.get_channels(subject, experiment, probe, "white_matter", asarray=False)
    
    bin_paths = ea.get_lfp_bin_paths(subject, experiment, alias, probe=probe)
    spg_paths = ea.get_analysis_counterparts(bin_paths, "tbd_spg.nc", subject)
    
    for bin_path, spg_path in zip(bin_paths, spg_paths):
        cx = load_trigger(bin_path, cx_chans)
        wm = load_trigger(bin_path, wm_chans)

        cx_wm_ref = cx - wm.values

        spgs = xr.Dataset({'cx_wm_ref': get_spectrogram(cx_wm_ref), 
                           'cx_cbm_ref': get_spectrogram(cx)})
            
        Path(spg_path).parent.mkdir(parents=True, exist_ok=True) # Create parent directories if they do not already exist.
        spgs.to_netcdf(spg_path)
        spgs.close()
        
        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"{current_time}: Finished {str(bin_path)}")

In [9]:
get_condition_spgs("Adrian", "sleep_homeostasis", "circadian_match", "imec1")

nChan: 385, nFileSamp: 18000082
nChan: 385, nFileSamp: 18000082
17:39:16: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX11-Adrian/8-30-2021/SpikeGLX/8-30-2021_g0/8-30-2021_g0_imec1/8-30-2021_g0_t0.imec1.lf.bin
nChan: 385, nFileSamp: 18000083
nChan: 385, nFileSamp: 18000083
17:45:24: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX11-Adrian/8-30-2021/SpikeGLX/8-30-2021_g0/8-30-2021_g0_imec1/8-30-2021_g0_t1.imec1.lf.bin
nChan: 385, nFileSamp: 18000083
nChan: 385, nFileSamp: 18000083
17:49:28: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX11-Adrian/8-30-2021/SpikeGLX/8-30-2021_g0/8-30-2021_g0_imec1/8-30-2021_g0_t2.imec1.lf.bin
nChan: 385, nFileSamp: 18000083
nChan: 385, nFileSamp: 18000083
17:54:11: Finished /Volumes/neuropixel_archive/Data/chronic/CNPIX11-Adrian/8-30-2021/SpikeGLX/8-30-2021_g0/8-30-2021_g0_imec1/8-30-2021_g0_t3.imec1.lf.bin
nChan: 385, nFileSamp: 18000082
nChan: 385, nFileSamp: 18000082
18:02:43: Finished /Volumes/neuropixel_archive/Data/chronic/